In [ ]:
# import shutil
# shutil.rmtree('/content/train', ignore_errors=True)
# shutil.rmtree('/content/test', ignore_errors=True)
# shutil.rmtree('/content/cats-v-dogs', ignore_errors=True)

In [1]:
from IPython.display import clear_output
!pip install --upgrade kaggle
clear_output()
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 70 bytes


In [3]:
!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition

 99% 805M/814M [00:02<00:00, 315MB/s]
100% 814M/814M [00:02<00:00, 287MB/s]


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!unzip -qx /content/dogs-vs-cats-redux-kernels-edition.zip

In [6]:
!unzip -qx /content/train.zip
!unzip -qx /content/test.zip

In [ ]:
import os
from glob import glob
from tqdm.notebook import tqdm
from shutil import copy
import pandas as pd
import cv2

In [ ]:
all_dogs = glob('/content/train/dog*')
all_cats= glob('/content/train/cat*')
all_dogs = sorted(all_dogs)
all_cats = sorted(all_cats)
print('Count of all dogs: ' ,len(all_dogs))
print('Count of all cats: ' ,len(all_cats))

Count of all dogs:  12500
Count of all cats:  12500


In [ ]:
all_images = glob('/content/train/*')

In [ ]:
print(len(all_images))

25000


In [ ]:
train_dir = '/content/cats-v-dogs/training/'
test_dir = '/content/cats-v-dogs/testing/'
to_create = [
            train_dir,
            test_dir,
]

for dir in to_create:
  os.makedirs(dir, exist_ok=True)

In [ ]:
from sklearn.model_selection import train_test_split
training_images, testing_images = train_test_split(all_images, test_size=0.1, random_state=101)

In [ ]:
print(len(training_images))
print(len(testing_images))

22500
2500


In [ ]:
print(train_dir)
for name in tqdm(training_images):
   copy(name , os.path.join(train_dir , name.split('/')[-1]))
print(test_dir)
for name in tqdm(testing_images):
   copy(name , os.path.join(test_dir , name.split('/')[-1]))

/content/cats-v-dogs/training/


  0%|          | 0/22500 [00:00<?, ?it/s]

/content/cats-v-dogs/testing/


  0%|          | 0/2500 [00:00<?, ?it/s]

In [ ]:
print(len(os.listdir(train_dir)))
print(len(os.listdir(test_dir)))

22500
2500


## prepare dataset

In [ ]:
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from torchvision import transforms
import torch
# Dataset
class CatsDogs(Dataset):
  def __init__(self, directory = '/content/cats-v-dogs/testing' , transform = None):
    super().__init__()
    self.names = glob(directory+'/*.jpg')[:]
    self.labels = []
    self.transform = transform
    
    for name in self.names: 
      if 'cat.' in name:
        self.labels.append(0)
      else:
        self.labels.append(1)
    self.labels = torch.tensor(self.labels, dtype = torch.float32)
    self.labels = torch.unsqueeze(self.labels, 1)

  def __len__(self):
    return (len(self.names))
  
  def __getitem__(self, idx):
    x = Image.open(self.names[idx])
    x = self.transform(x)
    y = self.labels[idx]
    return x,y

In [ ]:
train_transforms = transforms.Compose([
                          transforms.ToTensor(),
                          transforms.Resize((160,160)),
                          transforms.Normalize(mean=([0.485, 0.456, 0.406]),std=([0.229, 0.224, 0.225])),
                          

])

test_transforms = transforms.Compose([
                          transforms.ToTensor(),
                          transforms.Resize((160,160)),
                          transforms.Normalize(mean=([0.485, 0.456, 0.406]),std=([0.229, 0.224, 0.225]))
])

In [ ]:
train_dataset = CatsDogs(directory='/content/cats-v-dogs/training', transform=train_transforms)
test_dataset = CatsDogs(directory='/content/cats-v-dogs/testing', transform=test_transforms)

In [ ]:
for img in train_dataset:

  if img[0].shape[0] != 3:
    print (img[0].shape)


In [ ]:
train_loader = DataLoader(train_dataset, batch_size=64, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, num_workers=2)

In [ ]:
train_loader_it = iter(train_loader)

In [ ]:
print(next(train_loader_it)[0].shape)
print(next(train_loader_it)[1].shape)

torch.Size([64, 3, 160, 160])
torch.Size([64, 1])


## train model

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from time import time

In [ ]:
import torch
from torch import nn


class My_Classifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=3)
    self.conv2 = nn.Conv2d(in_channels=10, out_channels=16, kernel_size=3)

    self.conv3 = nn.Conv2d(in_channels=16, out_channels=20, kernel_size=3)
    self.conv4 = nn.Conv2d(in_channels=20, out_channels=28, kernel_size=3)

    self.conv5 = nn.Conv2d(in_channels=28, out_channels=32, kernel_size=3)
    self.conv6 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)

    self.maxpool = nn.MaxPool2d(kernel_size=2)
    self.relu = nn.ReLU()
    
    self.flatten = nn.Flatten()

    self.fc1 = nn.Linear(in_features=16384, out_features=128)
    self.fc2 = nn.Linear(in_features=128, out_features=1)
    
    self.dropout = nn.Dropout2d(0.2)

  def forward(self, input):
    #print(input.shape)
    x = self.conv1(input)
    x = self.relu(x)
    #print(x.shape)

    x = self.conv2(x)
    x = self.relu(x)
    #print(x.shape)

    x = self.maxpool(x)
    x = self.dropout(x)

    x = self.conv3(x)
    x = self.relu(x)
    #print(x.shape)

    x = self.conv4(x)
    x = self.relu(x)
    #print(x.shape)

    x = self.maxpool(x)
    x = self.dropout(x)

    x = self.conv5(x)
    x = self.relu(x)
    #print(x.shape)

    x = self.conv6(x)
    x = self.relu(x)
    #print(x.shape)

    x = self.maxpool(x)
    x = self.dropout(x)
    #print(x.shape)
    x = self.flatten(x)
    #print(x.shape)

    x = self.fc1(x)
    x = self.relu(x)
    #print(x.shape)

    y = self.fc2(x)



    return y

In [ ]:
mymodel =  My_Classifier()

In [ ]:
device = ('cuda' if torch.cuda.is_available else 'cpu')
mymodel =mymodel.to(device)
print(device)

cuda


In [ ]:
sig = nn.Sigmoid()
def binary_acc(y_pred, y_test):
    y_pred_2 = torch.tensor([0 if sig(i)< 0.5 else 1 for i in y_pred]).to(device)
    correct_result_sum = (y_pred_2==y_test).sum().float()
    return correct_result_sum.item()

In [ ]:
epochs = 30
criterion = nn.BCEWithLogitsLoss()
opt = torch.optim.Adam(mymodel.parameters())
losses = [] 
val_losses = []
train_accuracy = []
val_accuracy = []

for epoch in range(epochs):
  
  t_loss = []
  v_loss = []
  t_acc = 0
  v_acc = 0
  
  tic = time()

  mymodel.train()

  for batch in train_loader:
    y_pred = mymodel.forward(batch[0].to(device))
    opt.zero_grad()
    # print(y_pred[0])
    # print(batch[1][0])
    loss = criterion(y_pred, batch[1].to(device))
    t_loss.append(loss.item())
    loss.backward()
    opt.step()  
    acc = binary_acc(y_pred, batch[1].to(device)) 
    t_acc += acc 
  losses.append(np.mean(t_loss)) 
  train_accuracy.append(t_acc/len(train_dataset))

  mymodel.eval()
  for batch in test_loader:
    y_pred = mymodel.forward(batch[0].to(device))
    with torch.no_grad():
      loss = criterion(y_pred, batch[1].to(device))
      v_loss.append(loss.item())  
      acc = binary_acc(y_pred, batch[1].to(device)) 
      v_acc += acc 
  val_losses.append(np.mean(v_loss)) 
  val_accuracy.append(v_acc/len(test_dataset))

  toc = time()
  total_time =  toc - tic

  line = f'Epoch {epoch}, loss : {losses[-1]} , accuracy : {train_accuracy[-1]} , val_loss : {val_losses[-1]} , val_accuracy : {val_accuracy[-1]} , time : {total_time}'
  print(line)

Epoch 0, loss : 0.6372946087609638 , accuracy : 32.07351111111111 , val_loss : 0.5834151275455952 , val_accuracy : 32.2408 , time : 92.08221316337585
Epoch 1, loss : 0.5838766066696156 , accuracy : 32.114488888888886 , val_loss : 0.5351719066500664 , val_accuracy : 32.2776 , time : 92.10483002662659
Epoch 2, loss : 0.5153897905891592 , accuracy : 32.20497777777778 , val_loss : 0.45388455837965014 , val_accuracy : 32.3304 , time : 92.88597536087036
Epoch 3, loss : 0.44881601199846377 , accuracy : 32.20711111111111 , val_loss : 0.4183854855597019 , val_accuracy : 32.2784 , time : 91.27652788162231
Epoch 4, loss : 0.40661643479358067 , accuracy : 32.26488888888889 , val_loss : 0.4034545049071312 , val_accuracy : 32.3264 , time : 92.98845601081848
Epoch 5, loss : 0.36882998823950236 , accuracy : 32.27982222222222 , val_loss : 0.3723465662449598 , val_accuracy : 32.3424 , time : 93.71152234077454
Epoch 6, loss : 0.3327286597341299 , accuracy : 32.29342222222222 , val_loss : 0.36405762434005